In [70]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt



import os


In [71]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras

In [72]:
# #cats = r'\D:\downloads\Module 3_ Convolutional Neural Networks\Data set, Presentation and Python Notebook for Image recognition project module\train\cats'
# train_dir = r'D:\downloads\Module 3_ Convolutional Neural Networks\Data set, Presentation and Python Notebook for Image recognition project module\train'
# #test_dir =r'\Test'
# #dogs =r'\D:\downloads\Module 3_ Convolutional Neural Networks\Data set, Presentation and Python Notebook for Image recognition project module\train\dogs' 



# # for i in cats:   
# #     image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb',
# #     target_size= (28,28))
# #     image=np.array(image)
# #     data.append(image)
# #     labels.append(0)
# # for i in dogs:   
# #     image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb',
# #     target_size= (280,280))
# #     image=np.array(image)
# #     data.append(image)
# #     labels.append(1)
# # data = np.array(data)
# # labels = np.array(labels)



# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(train_generator, test_size=0.2,random_state=42)

In [91]:
import os
import numpy as np
from PIL import Image

# Define the directory containing the images
data_dir = r'D:\downloads\Module 3_ Convolutional Neural Networks\Data set, Presentation and Python Notebook for Image recognition project module\train'

# Define the classes
classes = ['cats', 'dogs']

# Define the target size of the images
target_size = (224, 224)

# Initialize empty lists for images and labels
images = []
labels = []

# Loop through the folders in the data directory
for class_name in classes:
    class_dir = os.path.join(data_dir, class_name)
    for filename in os.listdir(class_dir):
        if filename.endswith('.jpg'):  # check if the file is an image
            # Load the image and resize it
            img = Image.open(os.path.join(class_dir, filename)).resize(target_size)
            images.append(np.array(img))
            if class_name == 'cats':
                labels.append(0)
            elif class_name == 'dogs':
                labels.append(1)

# Convert the image and label lists to numpy arrays
images = np.array(images)
labels = np.array(labels)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images,labels ,test_size=0.2,random_state=42)
y_train =y_train.reshape(1,-1)
y_test=y_test.reshape(1,-1)
y_train.shape


(1, 1600)

In [109]:
train_x_flatten = X_train.reshape(X_train.shape[0], -1).T   # "-1"  reshape flatten the remaining dimensions
test_x_flatten = X_test.reshape(X_test.shape[0], -1).T

X_train_n = train_x_flatten/255.
X_test_n = test_x_flatten/255.
# print ("train_x's shape: " + str(X_train.shape))
# print ("test_x's shape: " + str(X_test.shape))
# test_x_flatten
# train_x_flatten.shape
X_train_n.shape
X_test_n.shape

(150528, 400)

In [93]:
def sigmoid(Z):
    A = 1/(1+np.exp(-Z))
    return A

def softmax(z):
    expZ = np.exp(z)
    return expZ/(np.sum(expZ, 0))

def relu(Z):
    A = np.maximum(0,Z)
    return A
def tanh(x):
  return np.tanh(x)    

In [94]:
y_test.shape

(1, 400)

In [95]:
print(y_train.shape)

(1, 1600)


In [96]:
y_test.shape[0]

1

In [97]:
def initialize_parameters(layer_dims):
    
    parameters = {} 
    L = len(layer_dims)            

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1])
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
    return parameters

In [98]:
layer_dims = [X_train_n.shape[0], 100, 200, 1]
params = initialize_parameters(layer_dims)

for l in range(1, len(layer_dims)):
    print("Shape of W" + str(l) + ":", params['W' + str(l)].shape)
    print("Shape of B" + str(l) + ":", params['b' + str(l)].shape, "\n")

Shape of W1: (100, 150528)
Shape of B1: (100, 1) 

Shape of W2: (200, 100)
Shape of B2: (200, 1) 

Shape of W3: (1, 200)
Shape of B3: (1, 1) 



In [99]:
def forward_propagation(X, parameters, activation):
   
    forward_cache = {}
    L = len(parameters) // 2                  
    
    forward_cache['A0'] = X

    for l in range(1, L):
        forward_cache['Z' + str(l)] = parameters['W' + str(l)].dot(forward_cache['A' + str(l-1)]) + parameters['b' + str(l)]
        
        if activation == 'tanh':
            forward_cache['A' + str(l)] = tanh(forward_cache['Z' + str(l)])
        else:
            forward_cache['A' + str(l)] = relu(forward_cache['Z' + str(l)])
            

    forward_cache['Z' + str(L)] = parameters['W' + str(L)].dot(forward_cache['A' + str(L-1)]) + parameters['b' + str(L)]
    
    if forward_cache['Z' + str(L)].shape[0] == 1:
        forward_cache['A' + str(L)] = sigmoid(forward_cache['Z' + str(L)])
    else :
        forward_cache['A' + str(L)] = softmax(forward_cache['Z' + str(L)])

   
    
    return forward_cache['A' + str(L)], forward_cache

In [100]:
aL, forw_cache = forward_propagation(X_train_n, params, 'relu')

for l in range(len(params)//2 + 1):
    print("Shape of A" + str(l) + " :", forw_cache['A' + str(l)].shape)

Shape of A0 : (150528, 1600)
Shape of A1 : (100, 1600)
Shape of A2 : (200, 1600)
Shape of A3 : (1, 1600)


In [101]:
def derivative_relu(Z):
    return np.array(Z > 0, dtype = 'float')

def derivative_tanh(x):
    return (1 - np.power(x, 2))

In [102]:
def compute_cost(AL, Y):
    m = Y.shape[0]
    
    if Y.shape[0] == 1:
        cost = (1./m) * (-np.dot(Y,np.log(AL).T) - np.dot(1-Y, np.log(1-AL).T))
    else:
        cost = -(1./m) * np.sum(Y * np.log(AL))
        
    cost = np.squeeze(cost)   
    print(cost)   # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    
    return cost

In [103]:
def backward_propagation(AL, Y, parameters, forward_cache, activation):
    
    grads = {}
    L = len(parameters)//2
    m = AL.shape[1]
    
    grads["dZ" + str(L)] = AL - Y
    grads["dW" + str(L)] = 1./m * np.dot(grads["dZ" + str(L)],forward_cache['A' + str(L-1)].T)
    grads["db" + str(L)] = 1./m * np.sum(grads["dZ" + str(L)], axis = 1, keepdims = True)
    
    for l in reversed(range(1, L)):
        if activation == 'tanh':
            grads["dZ" + str(l)] = np.dot(parameters['W' + str(l+1)].T,grads["dZ" + str(l+1)])*derivative_tanh(forward_cache['A' + str(l)])
        else:
            grads["dZ" + str(l)] = np.dot(parameters['W' + str(l+1)].T,grads["dZ" + str(l+1)])*derivative_relu(forward_cache['A' + str(l)])
            
        grads["dW" + str(l)] = 1./m * np.dot(grads["dZ" + str(l)],forward_cache['A' + str(l-1)].T)
        grads["db" + str(l)] = 1./m * np.sum(grads["dZ" + str(l)], axis = 1, keepdims = True)

    return grads

In [104]:
grads = backward_propagation(forw_cache["A" + str(3)], y_train, params, forw_cache, 'relu')

for l in reversed(range(1, len(grads)//3 + 1)):
    print("Shape of dZ" + str(l) + " :", grads['dZ' + str(l)].shape)
    print("Shape of dW" + str(l) + " :", grads['dW' + str(l)].shape)
    print("Shape of dB" + str(l) + " :", grads['db' + str(l)].shape, "\n")

Shape of dZ3 : (1, 1600)
Shape of dW3 : (1, 200)
Shape of dB3 : (1, 1) 

Shape of dZ2 : (200, 1600)
Shape of dW2 : (200, 100)
Shape of dB2 : (200, 1) 

Shape of dZ1 : (100, 1600)
Shape of dW1 : (100, 150528)
Shape of dB1 : (100, 1) 



In [105]:
def update_parameters(parameters, grads, learning_rate):

    L = len(parameters) // 2 
    
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
        
    return parameters

In [106]:
def predict(X, y, parameters, activation):

    m = X.shape[1]
    y_pred, caches = forward_propagation(X, parameters, activation)
    
    if y.shape[0] == 1:
        y_pred = np.array(y_pred > 0.5, dtype = 'float')
    else:
        y = np.argmax(y, 0)
        y_pred = np.argmax(y_pred, 0)
    
    return np.round(np.sum((y_pred == y)/m), 2)

In [107]:
def model(X, Y, layers_dims, learning_rate = 0.03, activation = 'relu', num_iterations = 3000):#lr was 0.009

    np.random.seed(1)
    costs = []              
    
    parameters = initialize_parameters(layers_dims)

    for i in range(0, num_iterations):

        AL, forward_cache = forward_propagation(X, parameters, activation)
        print(Y.shape)
        cost = compute_cost(AL, Y)


        grads = backward_propagation(AL, Y, parameters, forward_cache, activation)

        parameters = update_parameters(parameters, grads, learning_rate)
        
        if i % (num_iterations/10) == 0:
            print("\niter:{} \t cost: {} \t train_acc:{} \t test_acc:{}".format(i, np.round(cost, 2), predict(X_train_n, y_train, parameters, activation), predict(X_test_n, y_test, parameters, activation)))
        
        if i % 10 == 0:
            print("==", end = '')

       
       
    return parameters

In [110]:
layers_dims = [X_train_n.shape[0], 100,100, y_train.shape[0]] #  4-layer model
lr = 0.0075
iters = 200

parameters = model(X_train_n, y_train, layers_dims, learning_rate = lr, activation = 'relu', num_iterations = iters)

(1, 1600)
1114.6610676283317

iter:0 	 cost: 1114.66 	 train_acc:0.5 	 test_acc:0.5
==(1, 1600)
1137.549281803495
(1, 1600)
1123.5454571995115
(1, 1600)
1106.392593671993
(1, 1600)
1104.4893655527098
(1, 1600)
1110.2272428741971
(1, 1600)
1102.2589983476537
(1, 1600)
1098.5685341512603
(1, 1600)
1095.7006662821839
(1, 1600)
1093.6231839408138
(1, 1600)
1093.854068615522
==(1, 1600)
1099.8198026033779
(1, 1600)
1142.7024403691184
(1, 1600)
1110.702092457262
(1, 1600)
1102.3991908159737
(1, 1600)
1099.6132553858351
(1, 1600)
1098.3645659571534
(1, 1600)
1102.9884607452968
(1, 1600)
1112.2357383755361
(1, 1600)
1105.2314826384338
(1, 1600)
1099.0134757514268

iter:20 	 cost: 1099.01 	 train_acc:0.57 	 test_acc:0.56
==(1, 1600)
1095.3023353834537
(1, 1600)
1090.8890558829125
(1, 1600)
1088.673487017592
(1, 1600)
1091.6621414798742
(1, 1600)
1118.0405304429103
(1, 1600)
1102.0316259945525
(1, 1600)
1088.1651150073637
(1, 1600)
1091.156366131175
(1, 1600)
1107.1671225137177
(1, 1600)
1096.33